In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import datetime as dt
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#Initializes Chrome web driver

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\taher\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


In [29]:
#Scrapes the site by nesting the year in the team for loop to iterate through year 2000-2020 first,
#then change teams when counter reaches 2021

def team_salary():
    
    #set teams up as dictionary to access the key for the url input
    #and the access the value for df legibility
    teams = {'ANA':'Angels','ARI':'Diamondbacks','ATL':'Braves','BAL':'Orioles','BOS':'Red Sox',
    'CHN':'Cubs','CHA':'White Sox','CIN':'Reds','CLE':'Indians','COL':'Rockies','DET':'Tigers',
    'HOU':'Astros','KCA':'Royals','LAN':'Dodgers','MIA':'Marlins','MIL':'Brewers','MIN':'Twins',
    'NYN':'Mets','NYA':'Yankees','OAK':'Athletics','PHI':'Phillies','PIT':'Pirates','SDN':'Padres',
    'SFN':'Giants','SEA':'Mariners','SLN':'Cardinals','TBA':'Rays','TEX':'Rangers','TOR':'Blue Jays','WAS':'Nationals'}
    data2 = []
    
    for key,value in teams.items():
        for year in range(2000,2021):
            
            #Try/Except AttributeError to prevent break if url doesn't exist for a specific year
            try:
                url = f'https://legacy.baseballprospectus.com/compensation/index.php?team={key}&cyear={year}'
                browser.visit(url)
                browser.is_element_present_by_css('div.dataTables_wrapper',wait_time = 1)
                html = browser.html
                html_soup = soup(html,'html.parser')

                #finds the table in the HTML containing the player salary information
                table = html_soup.find('table',attrs={'id' : 'cotsplayers_datagrid'})
                t_body = table.find('tbody')
                rows = t_body.find_all('tr')

                #retrieves data from the entire table 
                for row in rows:
                    cols = row.find_all('td')
                    cols = [ele.text.strip() for ele in cols]
                    #removes the last column (un-needed and throws off list length and triggers error when input into df)
                    cols = cols[:-1]

                    cols.append(value)
                    cols.append(year)
                    data2.append([ele for ele in cols if ele])

                    

            except AttributeError:
                print(f'No web page exists for {value} in {year}')

    df =  pd.DataFrame(data = data2, columns = ["Player", "Pos", "Salary", "Pct", "WARP", "WARP/$M", "$/WARP","TEAM","YEAR"])
    return df
    


In [30]:

team_salary().to_csv('player_salaries.csv')

No web page exists for Marlins in 2000
No web page exists for Marlins in 2001
No web page exists for Marlins in 2002
No web page exists for Marlins in 2003
No web page exists for Marlins in 2004
No web page exists for Marlins in 2005
No web page exists for Marlins in 2006
No web page exists for Marlins in 2007
No web page exists for Marlins in 2008
No web page exists for Marlins in 2009
No web page exists for Marlins in 2010
No web page exists for Marlins in 2011
No web page exists for Nationals in 2000
No web page exists for Nationals in 2001
No web page exists for Nationals in 2002
No web page exists for Nationals in 2003
No web page exists for Nationals in 2004


In [37]:
#Scrapes the team's payroll using similar process in the team_salary() function

def payroll():
    teams = {'ANA':'Angels','ARI':'Diamondbacks','ATL':'Braves','BAL':'Orioles','BOS':'Red Sox',
    'CHN':'Cubs','CHA':'White Sox','CIN':'Reds','CLE':'Indians','COL':'Rockies','DET':'Tigers',
    'HOU':'Astros','KCA':'Royals','LAN':'Dodgers','MIA':'Marlins','MIL':'Brewers','MIN':'Twins',
    'NYN':'Mets','NYA':'Yankees','OAK':'Athletics','PHI':'Phillies','PIT':'Pirates','SDN':'Padres',
    'SFN':'Giants','SEA':'Mariners','SLN':'Cardinals','TBA':'Rays','TEX':'Rangers','TOR':'Blue Jays','WAS':'Nationals'}
    team_data = []
    
    for key,value in teams.items():
        url = f'https://legacy.baseballprospectus.com/compensation/index.php?team={key}&cyear=2019'
        browser.visit(url)
        browser.is_element_present_by_css('div.dataTables_wrapper',wait_time = 1)
        html = browser.html
        html_soup = soup(html,'html.parser')

        #finds the franchise payroll table in the HTML
        team_payroll_table = html_soup.find('table',attrs={'id':'cotspayroll_datagrid'})
        table_body = team_payroll_table.find('tbody')
        rows = table_body.find_all('tr')
        

        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            cols.append(value)
            team_data.append([ele for ele in cols if ele])


    df =  pd.DataFrame(data = team_data, columns = ["Year","Team Payroll", "Avg Payroll","Diff",'Team'])
    return df

In [38]:
payroll().to_csv('team_payrolls.csv')